# Prueba tecnica - BAC
## Pablo Escobar - 2024

### Pregunta 1

Dada esta base de datos, ¿cuál es el nivel de granularidad más fino en la reportería que se 
puede tener? Explique su razonamiento (el identificador de gestión no se usa en reportería).

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# install pandasql
%pip install pandasql

from pandasql import sqldf

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\python311\lib\site-packages\wkhtmltopdf-master-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: tensorview 0.4.1 has a non-standard dependency specifier pyecharts-snapshot>=0.1.10tensorflow>=2.0.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorview or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: wkhtmltopdf master has a non-sta

In [64]:
data1 = pd.read_csv('./databases/bdp1.csv', index_col=0)
data1.head()

,ID_cliente,Cuenta,Tarjeta
Id_gestión,,,
1,A,1,T1
2,A,1,T1
3,B,2,T2
4,B,2,T2
5,B,3,T3


Al observar la tabla podemos ver que un cliente puede tener muchas cuentas y que una cuenta puede tener muchas tarjetas. por lo que tenemos que la relación de cliente a cuenta es de uno a muchos y que la relación de cuenta a tarjeta es  de igual manera de una a muchas.

Al hablar de cuál es el nivel de granularidad más fino, estamos hablando de encontrar la combinación de campos que nos permita encontrar la mayor diferenciación entre los registros, en otras palabras, la manera de tener menos duplicidad. Para lograr esto debemos tomar en cuenta las relaciones que ya mencionamos anteriormente. Al ser las relaciones de una a muchas, la manera en que podemos obentener la menor duplicidad es usando la combinación de 
#### ID_Cliente + Cuenta + Tarjeta
Este nivel de combinación parece ser el más específico, ya que combina la información de cliente, cuenta y tarjeta, lo que debería garantizar la unicidad (o casi) de cada registro en la mayoría de los casos.

### Pregunta 2

Si se tienen las siguientes bases de datos, haga el código SQL que permita unir estas dos 
tablas de la forma más eficiente posible. Explique su razonamiento.

In [28]:
datap2_1 = pd.read_csv('./databases/p2/bdp2-1.csv')
datap2_2 = pd.read_csv('./databases/p2/bdp2-2.csv')

In [29]:
datap2_1.head()

,Cuenta,Tarjeta,Fecha,Monto
0,1,A1,202102,324.27
1,1,A1,202102,948.15
2,1,A1,202102,590.57
3,1,A2,202102,503.55
4,2,B1,202102,685.66


In [30]:
datap2_2.head()

,Cuenta,Tarjeta,Fecha,Monto
0,1,A1,202101,497.05
1,1,A1,202101,802.77
2,2,B1,202101,0.41
3,2,B1,202101,753.65
4,2,B1,202101,342.07


In [78]:
%%script false --no-raise-error
SELECT d1.cuenta, d1.tarjeta, d1.fecha, d1.monto 
FROM datap2_1 as d1
        UNION ALL
SELECT d2.cuenta, d2.tarjeta, d2.fecha, d2.monto 
FROM datap2_2 as d2
        ORDER BY d1.cuenta;

Couldn't find program: 'false'


In [79]:
print(sqldf('''
        SELECT d1.cuenta, d1.tarjeta, d1.fecha, d1.monto FROM datap2_1 as d1
        UNION ALL
        SELECT d2.cuenta, d2.tarjeta, d2.fecha, d2.monto FROM datap2_2 as d2
        ORDER BY d1.cuenta;
'''))

    Cuenta Tarjeta   Fecha   Monto
0        1      A1  202102  324.27
1        1      A1  202102  948.15
2        1      A1  202102  590.57
3        1      A2  202102  503.55
4        1      A1  202101  497.05
5        1      A1  202101  802.77
6        2      B1  202102  685.66
7        2      B1  202102  502.23
8        2      B1  202102  855.73
9        2      B1  202101    0.41
10       2      B1  202101  753.65
11       2      B1  202101  342.07
12       2      B1  202101  574.72
13       3      C1  202102  699.11
14       3      C1  202102  784.01
15       3      C1  202101  621.92
16       3      C2  202101  317.52
17       4      D1  202102  990.07
18       4      D1  202102  723.55
19       4      D1  202102  585.19
20       4      D2  202102  993.21
21       4      D1  202101  681.45
22       4      D2  202101  479.97
23       5      E1  202101  419.34
24       5      E1  202101  944.46
25       5      E1  202101  692.59


Dado que ambas tablas almacenan la misma información y no se trata de una relación con una tabla de catálogo, lo más lógico es unir las dos tablas utilizando un UNION. En este caso específico, es preferible usar UNION ALL, ya que debido a la granularidad de los datos, podría haber registros repetidos. Por ejemplo, si Pepito realiza dos transacciones de Q50, una a su papá y otra a su mamá, a nivel de tabla habría registros repetidos, ya que existiría la misma fila dos veces. Sin embargo, en realidad, se trata de dos transacciones separadas.

### Pregunta 3

Dada la estructura de esta base de datos, ¿qué variables (mínimo 1 y máximo 2) usaría para 
crearle un índice? Explique su razonamiento.

In [32]:
bdp3 = pd.read_csv('./databases/bdp3.csv')
bdp3

,Campo,Descripción,tipo de dato,Ejemplo,Cantidad únicos
0,ID_cliente,Identificador del cliente,string,CLIENTE1,113456
1,Fecha,Año y mes de los datos,numérico,202103,36
2,Cuenta,Número de cuenta que hizo la transacción,string,1234567890,185678
3,Tipo_cuenta,Cuenta es de crédito o débito,string,CRE - DEB,2
4,plan_lealtad,plan de lealtad de la cuenta,string,Millas Plus,12
5,Color,color de la tarjeta de crédito,string,Black,4
6,Monto_trx,Monto de la transacción,numérico,117.56,81456662
7,Afiliado,Comercio donde se hizo la transacción,string,30405060,25678


Usar un índice compuesto de ID_cliente y Cuenta es una buena opción porque permite realizar búsquedas más eficientes y específicas en la base de datos, mejorando significativamente el rendimiento en consultas que requieren filtrar tanto por cliente como por cuenta.  Además, aunque el índice compuesto ofrece ventajas significativas, los índices separados en ID_cliente y Cuenta también son útiles, ya que permiten optimizar las consultas individuales basadas únicamente en clientes o en cuentas, proporcionando flexibilidad y eficiencia en diversas operaciones de búsqueda y filtrado en la tabla.

### Pregunta 4

 ¿Cuál es la diferencia entre un Data Warehouse y un Data Mart? Si el jefe comercial de 
seguros le solicita un repositorio de datos, ¿cuál de estos 2 sería la mejor solución? Explique 
su razonamiento

Para explicar la diferencia entre Data Warehouse y data mart tambien es necesario entender el concepto de data lake. Un data lake es toda la información cruda sin transformar. Un data Warehouse ya posee datos ordenados y estructurados y un data mart es ya una separación del data warehouse, donde encontraremos información especifica de un campo en concreto. Un ejemplo es que el data warehouse es un almacen con diferentes productos para tiendas, tales como Comidas, Bebidas, Ropa, Juguetes etc. Un data mart vendría a ser una tienda que vende unicamente ROPA que sale del data warehouse.

Si el jefe comercial solicita un repositorio de datos, la respuesta correcta sería, que datos son los que necesita, ya que si es data que entra en una sola categoría, la respuesta sería un data mart.

### Pregunta 5

Se tienen varios exceles con la información que se muestra a continuación. Sugiera qué estructura tiene que tener una base de datos de SQL que unifique todas estas tablas. Explique su razonamiento (puede apoyarse creando un prototipo de cómo será la base de datos resultante). Cada fin de mes se irán agregando registros nuevos. Aparte de los 3 ejemplos hay 27 tablas más para unificar

In [33]:
bdp5_1 = pd.read_csv('./databases/p5/bdp5-1.csv')
bdp5_2 = pd.read_csv('./databases/p5/bdp5-2.csv')
bdp5_3 = pd.read_csv('./databases/p5/bdp5-3.csv')

In [34]:
bdp5_1

,ID_Cliente,Fecha,Trx_total
0,CLIENTE01,202012,447.96
1,CLIENTE02,202012,585.15
2,CLIENTE03,202012,560.09
3,CLIENTE04,202012,79.03
4,CLIENTE05,202012,226.61
5,CLIENTE06,202012,250.05
6,CLIENTE07,202012,294.66
7,CLIENTE08,202012,293.62
8,CLIENTE09,202012,425.01
9,CLIENTE10,202012,447.77


In [35]:
bdp5_2

,ID_Cliente,Fecha,Trx_total
0,CLIENTE01,202101,494.87
1,CLIENTE03,202101,649.52
2,CLIENTE05,202101,531.75
3,CLIENTE06,202101,3.98
4,CLIENTE08,202101,499.74
5,CLIENTE09,202101,26.11
6,CLIENTE10,202101,939.41
7,CLIENTE14,202101,278.27
8,CLIENTE22,202101,28.83
9,CLIENTE77,202101,613.60


In [36]:
bdp5_3

,ID_Cliente,Fecha,Trx_total
0,CLIENTE01,202102,494.87
1,CLIENTE03,202102,278.27
2,CLIENTE04,202102,531.75
3,CLIENTE05,202102,3.98
4,CLIENTE07,202102,499.74
5,CLIENTE08,202102,26.11
6,CLIENTE09,202102,939.41
7,CLIENTE11,202102,28.83
8,CLIENTE23,202102,649.52
9,CLIENTE62,202102,613.60


Debido a que los datos son los mismos en todas las tablas, lo mejor sería centralizar esos datos en una sola tabla. De esta manera podremos tener centralizados los datos y al momento de hacer una consulta de transacciones tendremos la data en un solo lugar

In [77]:
%%script false --no-raise-error

CREATE TABLE Transactions  (
    ID_Transaccion SERIAL PRIMARY KEY, # Ya que es Primary key, ya es un campo indexado
    ID_Cliente varchar(10) NOT NULL, # asume que el ID del cliente es un string UUID o bien cambiarlo a INT de ser necesario 
    fecha DATE NOT NULL,
    Trx_total DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (ID_Cliente) REFERENCES Clientes(id)
);

Couldn't find program: 'false'


### Pregunta 6

Considere la base de datos que se muestra a continuación. A esta base de datos se le hizo una tabla índice con las siguientes variables: V1 y V2 pero no está funcionando como se esperaba. Encuentre el error.

In [67]:
dbp6 = pd.read_csv('./databases/p6/dbp6.csv')
dbp6

,V1,V2,V3,V4,V5
0,202001,Z,628.96,203.89,0.22
1,202001,Y,407.45,187.96,0.60
2,202001,X,23.29,91.17,0.86
3,202005,X,86.49,150.19,0.47
4,202005,Y,576.18,55.58,0.89
5,202005,Z,40.32,22.65,0.88
6,202101,Z,37.19,8.36,0.59
7,202101,Y,179.92,134.58,0.23
8,202101,X,109.67,25.65,0.40
9,201911,X,365.30,28.64,0.82


Si la tabla índice no está correctamente ordenada por las columnas V1 y V2, las búsquedas pueden ser ineficientes.

### Pregunta 7

Se tienen las 2 bases de datos que se muestran a continuación. A la base de datos A se le quiere agregar un campo que se encuentra en la tabla B. Sugiera 2 formas de hacer esta operación, una usando joins y la otra sin usarlos

In [38]:
bdp7_1 = pd.read_csv('./databases/p7/bdp7-1.csv')
bdp7_2 = pd.read_csv('./databases/p7/bdp7-2.csv')

In [68]:
bdp7_1

,Cliente_ID,Tipo_cliente,Cant_cuentas
0,CLIENTE01,CRE,1
1,CLIENTE02,CRE,2
2,CLIENTE03,CRE,3
3,CLIENTE04,CRE,1
4,CLIENTE05,CRE,2
5,CLIENTE06,CRE,2
6,CLIENTE07,CRE,3
7,CLIENTE08,CRE,1
8,CLIENTE09,CRE,4
9,CLIENTE10,CRE,1


In [69]:
bdp7_2

,Cliente_ID,Facturacion
0,CLIENTE01,147.02
1,CLIENTE02,215.09
2,CLIENTE03,711.89
3,CLIENTE04,672.37
4,CLIENTE05,596.86
5,CLIENTE06,658.09
6,CLIENTE07,554.31
7,CLIENTE08,394.17
8,CLIENTE09,433.99
9,CLIENTE10,961.11


### USANDO JOIN

In [80]:
%%script false --no-raise-error
SELECT d1.cliente_id, d1.tipo_cliente, d1.cant_cuentas, d2.facturacion
    FROM bdp7_1 as d1
        INNER JOIN bdp7_2 as d2
            ON d1.cliente_id = d2.cliente_id;

Couldn't find program: 'false'


In [39]:
print(sqldf('''
        SELECT d1.cliente_id, d1.tipo_cliente, d1.cant_cuentas, d2.facturacion
        FROM bdp7_1 as d1
        INNER JOIN bdp7_2 as d2
            ON d1.cliente_id = d2.cliente_id;
 '''))

   Cliente_ID Tipo_cliente  Cant_cuentas  Facturacion
0   CLIENTE01          CRE             1       147.02
1   CLIENTE02          CRE             2       215.09
2   CLIENTE03          CRE             3       711.89
3   CLIENTE04          CRE             1       672.37
4   CLIENTE05          CRE             2       596.86
5   CLIENTE06          CRE             2       658.09
6   CLIENTE07          CRE             3       554.31
7   CLIENTE08          CRE             1       394.17
8   CLIENTE09          CRE             4       433.99
9   CLIENTE10          CRE             1       961.11
10  CLIENTE11          CRE             1       274.90
11  CLIENTE12          CRE             2       455.11
12  CLIENTE13          CRE             1       689.29
13  CLIENTE14          CRE             2       498.56


### SIN USAR JOINS

In [81]:
%%script false --no-raise-error
SELECT 
    d1.cliente_id, 
    d1.tipo_cliente, 
    d1.cant_cuentas, 
        (SELECT d2.facturacion 
            FROM bdp7_2 AS d2 
            WHERE d2.cliente_id = d1.cliente_id) AS facturacion
    FROM bdp7_1 AS d1;

Couldn't find program: 'false'


In [40]:
print(sqldf(''' 
    SELECT 
        d1.cliente_id, 
        d1.tipo_cliente, 
        d1.cant_cuentas, 
            (SELECT d2.facturacion 
            FROM bdp7_2 AS d2 
            WHERE d2.cliente_id = d1.cliente_id) AS facturacion
    FROM bdp7_1 AS d1;
    '''))

   Cliente_ID Tipo_cliente  Cant_cuentas  facturacion
0   CLIENTE01          CRE             1       147.02
1   CLIENTE02          CRE             2       215.09
2   CLIENTE03          CRE             3       711.89
3   CLIENTE04          CRE             1       672.37
4   CLIENTE05          CRE             2       596.86
5   CLIENTE06          CRE             2       658.09
6   CLIENTE07          CRE             3       554.31
7   CLIENTE08          CRE             1       394.17
8   CLIENTE09          CRE             4       433.99
9   CLIENTE10          CRE             1       961.11
10  CLIENTE11          CRE             1       274.90
11  CLIENTE12          CRE             2       455.11
12  CLIENTE13          CRE             1       689.29
13  CLIENTE14          CRE             2       498.56


### Pregunta 8

Se tienen las siguientes 3 bases de datos. Escriba el código SQL que va a permitir tener en una sola tabla la información de Cuenta, plan de lealtad, calificación por mes y facturación total (por mes). Trate de hacer el query lo más eficiente posible

In [41]:
dbp8_1 = pd.read_csv('./databases/p8/bdp8-1.csv')
dbp8_2 = pd.read_csv('./databases/p8/bdp8-2.csv')

In [42]:
dbp8_1

,Cuenta,Plan_lealtad
0,1,MillasPlus
1,2,Cashback
2,3,Cashback
3,4,MillasPlus
4,5,MillasPlus
5,6,Cashback


In [70]:
dbp8_2

,Cuenta,Fecha,Calificacion
0,1,202101,A
1,1,202102,A
2,1,202103,A
3,2,202101,A
4,2,202102,C
5,2,202103,D
6,3,202101,E
7,3,202102,E
8,3,202103,E
9,4,202101,A


Para evitar la redundancia de datos crearemos una tabla catalogo para los diferentes planes de lealtad

In [49]:
planes_lealtad = pd.DataFrame({
    'id': [1, 2],
    'plan': ['MillasPlus', 'Cashback'],
})
planes_lealtad = planes_lealtad.set_index('id')
planes_lealtad

,plan
id,
1,MillasPlus
2,Cashback


Ahora uniremos las tablas

In [82]:
%%script false --no-raise-error
SELECT d1.cuenta, d2.calificacion, d3.id as plan_id,d3.plan
FROM dbp8_1 as d1
    INNER JOIN dbp8_2 as d2
        ON d1.cuenta = d2.cuenta
    INNER JOIN planes_lealtad as d3
        ON d1.plan_lealtad = d3.plan
           

Couldn't find program: 'false'


In [75]:
print(sqldf('''
            SELECT d1.cuenta, d2.calificacion, d3.id as plan_id,d3.plan
            FROM dbp8_1 as d1
            INNER JOIN dbp8_2 as d2
                ON d1.cuenta = d2.cuenta
            INNER JOIN planes_lealtad as d3
                ON d1.plan_lealtad = d3.plan
            '''))

    Cuenta Calificacion  plan_id        plan
0        1            A        1  MillasPlus
1        1            A        1  MillasPlus
2        1            A        1  MillasPlus
3        2            A        2    Cashback
4        2            C        2    Cashback
5        2            D        2    Cashback
6        3            E        2    Cashback
7        3            E        2    Cashback
8        3            E        2    Cashback
9        4            A        1  MillasPlus
10       4            A        1  MillasPlus
11       4            B        1  MillasPlus
12       5            E        1  MillasPlus
13       5            E        1  MillasPlus
14       5            E        1  MillasPlus
15       6            C        2    Cashback
16       6            C        2    Cashback
17       6            G        2    Cashback


### Mejorar por agrupación

In [83]:
%%script false --no-raise-error
SELECT d1.cuenta, d2.calificacion, d3.id as plan_id,d3.plan, COUNT(d1.cuenta) as cantidad
    FROM dbp8_1 as d1
        INNER JOIN dbp8_2 as d2
            ON d1.cuenta = d2.cuenta
        INNER JOIN planes_lealtad as d3
                ON d1.plan_lealtad = d3.plan
    GROUP BY d1.cuenta, d2.calificacion

Couldn't find program: 'false'


In [76]:
print(sqldf('''
            SELECT d1.cuenta, d2.calificacion, d3.id as plan_id,d3.plan, COUNT(d1.cuenta) as cantidad
            FROM dbp8_1 as d1
            INNER JOIN dbp8_2 as d2
                ON d1.cuenta = d2.cuenta
            INNER JOIN planes_lealtad as d3
                ON d1.plan_lealtad = d3.plan
            GROUP BY d1.cuenta, d2.calificacion
            '''))

   Cuenta Calificacion  plan_id        plan  cantidad
0       1            A        1  MillasPlus         3
1       2            A        2    Cashback         1
2       2            C        2    Cashback         1
3       2            D        2    Cashback         1
4       3            E        2    Cashback         3
5       4            A        1  MillasPlus         2
6       4            B        1  MillasPlus         1
7       5            E        1  MillasPlus         3
8       6            C        2    Cashback         2
9       6            G        2    Cashback         1
